In [1]:
#import statements

import pandas as pd
import numpy as np
import time
import os

import numpy as np


In [2]:
# merge data from all years
data_folder = '../classify_DRMIS_SAP_transactions_DATA/'
def absoluteFilePaths(directory):
   for dirpath,_,filenames in os.walk(directory):
       for f in filenames:
           yield os.path.abspath(os.path.join(dirpath, f))
#merge actuals from each fiscal year
A = list(absoluteFilePaths(data_folder+'act'))
for i in range(len(A)):
    A[i] = pd.read_csv(A[i],engine = 'python')
actuals = pd.concat(A, axis = 0, ignore_index = True)

#merge commitments from each fiscal year
B = list(absoluteFilePaths(data_folder+'comm'))
for i in range(len(B)):
    B[i] = pd.read_csv(B[i],engine = 'python')
commitments = pd.concat(B, axis = 0, ignore_index = True)

#merge consumption from each fiscal year
C = list(absoluteFilePaths(data_folder+'consum'))
for i in range(len(C)):
    C[i] = pd.read_csv(C[i],engine = 'python')
consumption = pd.concat(C, axis = 0, ignore_index = True)


In [4]:
# remove incorrect transactions, go through actuals, consumption and committments
# remove duplicated transactions
#consntants and functions
LNum_GL={1:2201,2:7208,3:7209,4:7210,5:7312,6:7341,7:9129,8:9219,9:9220,10:9335}
LNum = list(LNum_GL.keys())
GL = list(LNum_GL.values())

def LNum2GL(col):
    for i in range(len(col-1)):
        col[i]=LNum_GL[(col[i])]
    return(col)
        
#makes numbers from consumption match in commitments
def formnum(n):
    if '.' not in n[-3:]:
        return(n +'.00')
    else:
        return(n)
    
## inputs ##
#actuals = pd.read_csv('TEME Actuals.csv', engine = 'python')
#consumption = pd.read_csv('TEME Consumption.csv')
consumption_dummy = consumption.copy()
#commitments = pd.read_csv('TEME commitments.csv')
#output_file = 'transactions to fix13.csv'

# create output dataframe
errors = pd.DataFrame(columns = ['Trigger document','GL (consumption)','GL (actuals)','Desc','Cost',])

#format consumption
x = len(consumption)-1

for i in range(len(consumption['Document item'])-1):
    if pd.isnull(consumption['Document item'][i+1]) == True:
        consumption['Document item'][i+1] = consumption['Document item'][i] 

consumption_dummy['Document item'] = consumption['Document item']
consumption['GL']=LNum2GL(consumption_dummy['Document item']).copy()

#remove all nans from trigger document

for i in range(len(consumption['Trigger document'])-1):
    if pd.isnull(consumption['Trigger document'][i])==True:
        consumption.drop([i,i],axis=0,inplace=True)
consumption.drop([x,x],axis=0,inplace=True)

## format consumption ##

for i in range(len(actuals)-1):
    if pd.isnull(actuals['Ref. document number'][i])==True:
        actuals.drop([i,i],axis=0,inplace=True)

# columns for df of incorrect transactions

A=[] # cost
B=[] # GL (consumption)
C=[] # GL (actuals)
D=[] # transaction 
E=[] # description


for i in list(consumption.index.values):
# for a given transaction, if the "cost element" column in actuals does not match the "GL" column in consumption, that transaction is incorrect
# compare consumption to actuals
    x = consumption['Consptn in Doc.Crncy'][i]
    if consumption['Consptn in Doc.Crncy'][i] in list(actuals['Val/COArea Crcy']) and str(list(actuals[actuals['Val/COArea Crcy']==x]['Cost Element'])) != str(list(consumption[consumption['Consptn in Doc.Crncy']==x]['GL'])):
        A.append(consumption['Trigger document'][i])
        B.append(consumption['GL'][i])
        C.append(list(actuals[actuals['Val/COArea Crcy']==x]['Cost Element'])[0])
        D.append(consumption['Consptn in Doc.Crncy'][i])
        E.append(consumption['Reduction text'][i])

# for a given transaction, if the "cost element" column in commitments does not match the "GL" column in consumption, that transaction is incorrect
# compare consumption to commitments    
    if consumption['Consptn in Doc.Crncy'][i] in list(commitments['Val/COArea Crcy']) and str(list(consumption[consumption['Consptn in Doc.Crncy']==x]['GL'])) != str(list(commitments[commitments['Val/COArea Crcy']==formnum(x)]['Cost element'])): 
        A.append(consumption['Trigger document'][i])
        B.append(consumption['GL'][i])
        C.append(list(actuals[actuals['Val/COArea Crcy']==x]['Cost Element'])[0])
        D.append(consumption['Consptn in Doc.Crncy'][i])
        E.append(consumption['Reduction text'][i])
        
errors['Trigger document'] = A
errors['GL (consumption)'] = B
errors['GL (actuals)'] = C
errors['Cost'] = D
errors['Desc'] = E


#extract incorrect transactions, create df for them
for i in range(len(errors)):
    if errors['GL (consumption)'][i] == errors['GL (actuals)'][i]:
        errors.drop([i,i],axis=0,inplace=True)

#export output as csv
#errors.to_csv(output_file)

#remove incorrect transactions from consumption df
cond = consumption['Trigger document'].isin(errors['Trigger document'])

#consumption lists all correct transactions, from the fiscal years of 2017, 2018, 2019

consumption.drop(consumption[cond].index, inplace = True)

#corrected_data = consumption[['GL', 'Reduction text, Cost']]


c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [5]:
# prepped data for analysis
consumption.to_csv(data_folder+'input_data_base.csv')

In [6]:
consumption.head()

,Trigger document,Earmarked Funds,Document item,Business Transaction,Consptn in Doc.Crncy,Currency,Revaluation rate,Consumption in LC,Local Currency,Delete Date,Document Date,Reduction text,Purchasing Document,Reference document,Reference procedure,Reference line item,Item,Line item,Seq. No. of Account Assgt,GL
1,0014268084 00020,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-21,2018-03-21,Freight Charges,14268084.0,14268084.0,PREQ,20.0,20.0,NaN,NaN,2201.0
2,0014263945 00020,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-27,2018-03-19,Freight charges,14263945.0,14263945.0,PREQ,20.0,20.0,NaN,NaN,2201.0
3,0014263837 00020,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-28,2018-03-19,Freight,14263837.0,14263837.0,PREQ,20.0,20.0,NaN,NaN,2201.0
4,0014262000 00020,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-27,2018-03-16,Freight Charges,14262000.0,14262000.0,PREQ,20.0,20.0,NaN,NaN,2201.0
5,0014260003 00030,NaN,1.0,Purchase Requisition,0,CAD,NaN,0,CAD,2018-03-28,2018-03-15,Freight,14260003.0,14260003.0,PREQ,30.0,30.0,NaN,NaN,2201.0
